In [1]:
from pathlib import Path
import pandas as pd
import pyarrow as pa
import numpy as np
import re
from tqdm.notebook import tqdm
from datetime import datetime

In [2]:
def _save_df_to_zip(df_: pd.DataFrame, archive_name: str = 'archive', folder: str='data', replace: bool=False) -> None:
    # Путь к файлу
    file_path = Path(folder).joinpath(archive_name + '.zip')
    Path(folder).mkdir(exist_ok=True)
    # Проверяем, существует ли файл
    if file_path.exists() and not replace:
        # Получаем время создания файла
        time = datetime.fromtimestamp(file_path.lstat().st_atime).strftime('%Y-%m-%d %H-%M')

        # Создаем новое имя файла с добавлением времени Unix
        new_file_name = file_path.stem + " " + str(time) + file_path.suffix

        # Создаем новый путь для переименованного файла
        new_file_path = file_path.with_name(new_file_name)
        # Переименовываем файл
        file_path.rename(new_file_path)

# to csv
    compression_opts = dict(method='zip', archive_name=f'{archive_name}.csv')
    df_.to_csv(f'{folder}/{archive_name}.zip', index=False, compression=compression_opts, encoding='utf-8')

def _save_df_to_zip_for_excel(df_: pd.DataFrame, archive_name: str = 'archive', folder: str='data', replace: bool=False) -> None:
    # Путь к файлу
    file_path = Path(folder).joinpath(archive_name + '.zip')
    Path(folder).mkdir(exist_ok=True)
    # Проверяем, существует ли файл
    if file_path.exists() and not replace:
        # Получаем время создания файла
        time = datetime.fromtimestamp(file_path.lstat().st_atime).strftime('%Y-%m-%d %H-%M')

        # Создаем новое имя файла с добавлением времени Unix
        new_file_name = file_path.stem + " " + str(time) + file_path.suffix

        # Создаем новый путь для переименованного файла
        new_file_path = file_path.with_name(new_file_name)
        # Переименовываем файл
        file_path.rename(new_file_path)

# to csv
    compression_opts = dict(method='zip', archive_name=f'{archive_name}.csv')
    df_.to_csv(f'{folder}/{archive_name}.zip', sep=';', index=False, decimal=',', encoding='utf-8', compression=compression_opts)

In [3]:
p = Path('data')
if p.exists():
    dataframes_list = list(filter(lambda x: re.match(r'(result \d{4}.zip)', x.name), p.iterdir()))
dataframes_list

[WindowsPath('data/result 0000.zip'),
 WindowsPath('data/result 0001.zip'),
 WindowsPath('data/result 0002.zip'),
 WindowsPath('data/result 0003.zip'),
 WindowsPath('data/result 0004.zip'),
 WindowsPath('data/result 0005.zip'),
 WindowsPath('data/result 0006.zip'),
 WindowsPath('data/result 0007.zip'),
 WindowsPath('data/result 0008.zip'),
 WindowsPath('data/result 0009.zip'),
 WindowsPath('data/result 0010.zip'),
 WindowsPath('data/result 0011.zip'),
 WindowsPath('data/result 0012.zip'),
 WindowsPath('data/result 0013.zip'),
 WindowsPath('data/result 0014.zip'),
 WindowsPath('data/result 0015.zip'),
 WindowsPath('data/result 0016.zip'),
 WindowsPath('data/result 0017.zip'),
 WindowsPath('data/result 0018.zip'),
 WindowsPath('data/result 0019.zip'),
 WindowsPath('data/result 0020.zip'),
 WindowsPath('data/result 0021.zip'),
 WindowsPath('data/result 0022.zip'),
 WindowsPath('data/result 0023.zip'),
 WindowsPath('data/result 0024.zip'),
 WindowsPath('data/result 0025.zip'),
 WindowsPath

In [4]:
columns_dates = ['date', 'dateStart', 'dateEnd']
columns_types = {
                    'sity':pd.StringDtype(),
                    'market':pd.StringDtype(),
                    'discounts':pd.UInt32Dtype(),
                    'data_uuid':pd.StringDtype(),
                    'href':pd.StringDtype(),
                    'su1':pd.StringDtype(),
                    'su2':pd.StringDtype(),
                    'su3':pd.StringDtype(),
                    'title':pd.StringDtype(),
                    'sku_id':pd.StringDtype(),
                    'price':pd.UInt32Dtype(),
                    'price_from':pd.UInt32Dtype(),
                    'price_to':pd.UInt32Dtype(),
                    'discountPercent':pd.UInt32Dtype(),
                    'quantity':pd.Float32Dtype(),
                    'quantityUnit':pd.StringDtype(),
                }

In [5]:
df = list()
for item in tqdm(dataframes_list, desc='Load:'):
    # df.append(pd.read_csv(item, dtype_backend='pyarrow', engine='pyarrow', parse_dates=['date', 'dateStart', 'dateEnd'], dtype=columns_types))
    df.append(pd.read_csv(item, parse_dates=columns_dates, dtype=columns_types))
df = pd.concat(df, axis=0, ignore_index=True)

Load::   0%|          | 0/34 [00:00<?, ?it/s]

In [6]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 7419849 entries, 0 to 7419848
Data columns (total 19 columns):
 #   Column           Dtype         
---  ------           -----         
 0   date             datetime64[ns]
 1   sity             string        
 2   market           string        
 3   discounts        UInt32        
 4   data_uuid        string        
 5   href             string        
 6   su1              string        
 7   su2              string        
 8   su3              string        
 9   title            string        
 10  sku_id           string        
 11  dateStart        datetime64[ns]
 12  dateEnd          datetime64[ns]
 13  price            UInt32        
 14  price_from       UInt32        
 15  price_to         UInt32        
 16  discountPercent  UInt32        
 17  quantity         Float32       
 18  quantityUnit     string        
dtypes: Float32(1), UInt32(5), datetime64[ns](3), string(10)
memory usage: 948.2 MB


In [7]:
_save_df_to_zip_for_excel(df, 'result')

In [8]:
df.head()


,date,sity,market,discounts,data_uuid,href,su1,su2,su3,title,sku_id,dateStart,dateEnd,price,price_from,price_to,discountPercent,quantity,quantityUnit
0,2023-10-03 09:51:48.273719,Санкт-Петербург,Перекрёсток,1986,94d98417-42c3-11e6-9419-52540010b608,https://edadeal.ru/sankt-peterburg/retailers/p...,Авто,Автохимия и автокосметика,Ароматизаторы салона,Ароматизатор воздуха МАРКЕТ Organica Black Wat...,7c63c87f-2b7b-5bb5-af06-4c657b7f7499,2023-09-28,2023-10-25,<NA>,<NA>,16999,49,<NA>,<NA>
1,2023-10-03 09:51:48.273719,Санкт-Петербург,Перекрёсток,1986,94d98417-42c3-11e6-9419-52540010b608,https://edadeal.ru/sankt-peterburg/retailers/p...,Авто,Автохимия и автокосметика,Ароматизаторы салона,Ароматизатор воздуха МАРКЕТ BottleMania Black ...,a9a1a5fc-021b-5e5f-a915-6688afbed062,2023-09-28,2023-10-25,<NA>,<NA>,14999,26,<NA>,<NA>
2,2023-10-03 09:51:48.273719,Санкт-Петербург,Перекрёсток,1986,94d98417-42c3-11e6-9419-52540010b608,https://edadeal.ru/sankt-peterburg/retailers/p...,Авто,Автохимия и автокосметика,Ароматизаторы салона,Ароматизатор воздуха МАРКЕТ Wooden Aroma Gentl...,33329577-c5de-50af-85af-c2d084a3f837,2023-09-28,2023-10-25,<NA>,<NA>,11999,28,<NA>,<NA>
3,2023-10-03 09:51:48.273719,Санкт-Петербург,Перекрёсток,1986,94d98417-42c3-11e6-9419-52540010b608,https://edadeal.ru/sankt-peterburg/retailers/p...,Авто,Аксессуары и оборудование,<NA>,Ароматизатор воздуха МАРКЕТ Super Flower Black...,db207ea6-eed4-52a0-83b2-fb6901777f5c,2023-09-28,2023-10-25,<NA>,<NA>,24999,25,<NA>,<NA>
4,2023-10-03 09:51:48.273719,Санкт-Петербург,Перекрёсток,1986,94d98417-42c3-11e6-9419-52540010b608,https://edadeal.ru/sankt-peterburg/retailers/p...,Компьютерная техника,Аксессуары,"Кабели, разъемы, переходники",Кабель МАРКЕТ для передачи аудиосигнала модель...,111d7ebc-e168-5ed1-a4f1-23d207ca4652,2023-09-28,2023-10-25,<NA>,<NA>,11999,26,<NA>,<NA>


In [9]:
# _save_df_to_zip(df, 'result')
#  na_rep='null',
df.to_csv('a:/result.csv', sep=';', index=False, decimal=',', encoding='utf-8')

In [10]:
for item in df.select_dtypes('string').columns.to_list():
    print(item, df[item].str.contains(r'(.*\n.*)', regex=True).sum())

C:\Users\f999145\AppData\Local\Temp\ipykernel_6848\2835348144.py:2: UserWarning: This pattern is interpreted as a regular expression, and has match groups. To actually get the groups, use str.extract.
  print(item, df[item].str.contains(r'(.*\n.*)', regex=True).sum())


sity 0
market 0
data_uuid 0
href 0
su1 0
su2 0
su3 0
title 0
sku_id 0
quantityUnit 0
